# Lab 03 FastText

Installing Datasets Library

Installing FastText

Loading imdb dataset

In [1]:
from datasets import load_dataset_builder, load_dataset

ds_builder = load_dataset_builder("imdb")
dataset = load_dataset("imdb")

/Users/longvo/Library/Caches/pypoetry/virtualenvs/nlp1-je8b2UjA-py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset imdb (/Users/longvo/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 180.52it/s]


Spliting imdb dataset into training and testing datasets

In [2]:
train_data, test_data = load_dataset('imdb', split =['train', 'test'])

Found cached dataset imdb (/Users/longvo/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 163.23it/s]


## Question 1 : Turn the dataset into a dataset compatible with Fastext

Converting the training and testing datasets to pandas.

In [3]:
df_pandas_train = train_data.to_pandas()
df_pandas_test = test_data.to_pandas()

Pretreatment, applying lower casing and punctuation removal.

In [4]:
import string

def lower_case_rm_punc(dataframe):
  '''Applying lower casing and punctuation removal.

  Arg: 
      Dataframe

  Returns:
      Dataframe 
  '''
  dataframe['text'] = dataframe['text'].str.lower()
  dataframe['text'] = dataframe['text'].str.replace(r'[^\w\s]+', '')
  return dataframe

df_train = lower_case_rm_punc(df_pandas_train)
df_test = lower_case_rm_punc(df_pandas_test)

/var/folders/st/5wgrjpj9129bgxw_hskwwwlh0000gn/T/ipykernel_60338/3809465340.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['text'] = dataframe['text'].str.replace(r'[^\w\s]+', '')


Switching the columns label and text for both the training and testing datasets to respect the training file format.

Shuffling the data to avoid having a strong model bias toward negative, for both datasets.

In [5]:
df_train_reindex=df_train.reindex(columns=["label","text"])
df_train_reindex.sort_values(by=['label'], ascending=False, inplace=True)
df_test_reindex=df_test.reindex(columns=["label","text"])
df_test_reindex.sort_values(by=['label'], ascending=False, inplace=True)

Adding `__label__` to each line of the dataframe, to respect the training file format, and exporting it as a text (.txt) because training a FastText classifier takes a text file as input.

In [6]:
import numpy as np
np.savetxt(r'train.txt', df_train_reindex.values, fmt='__label__%s %s')
np.savetxt(r'test.txt', df_test_reindex.values, fmt='__label__%s %s')

## Question 2 : Train a FastText classifier with default parameters on the training data, and evaluate it on the test data using accuracy.

In [7]:
import fasttext

model = fasttext.train_supervised('train.txt')
model.test('test.txt')       

Read 5M words
Number of words:  121573
Number of labels: 2
Progress: 100.0% words/sec/thread: 4408987 lr:  0.000000 avg.loss:  0.359999 ETA:   0h 0m 0s


(25000, 0.87528, 0.87528)

## Question 3: Use the hyperparameters search functionality of FastText and repeat question 2

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Split train, validation
X_train, X_val, y_train, y_val = train_test_split(df_train_reindex['text'],df_train_reindex['label'],train_size=0.8)

# 2. Concatenate 
df_train = pd.concat([y_train,X_train],axis=1)
df_valid = pd.concat([y_val,X_val],axis=1)

# 3. Shuffling the datasets
df_train.sort_values(by=['label'], ascending=False, inplace=True)
df_valid.sort_values(by=['label'], ascending=False, inplace=True)

# 4. Export as .txt and respect the training file format
np.savetxt(r'imdb_train.txt', df_train.values, fmt='__label__%s %s')
np.savetxt(r'imdb_valid.txt', df_valid.values, fmt='__label__%s %s')

# 5. Create model by activating the hyperparameter optimization search functionality
model = fasttext.train_supervised(input='imdb_train.txt', autotuneValidationFile='imdb_valid.txt')

Progress: 100.0% Trials:   17 Best score:  0.886800 ETA:   0h 0m 0s
Training again with best arguments
Read 4M words
Number of words:  107837
Number of labels: 2
Progress: 100.0% words/sec/thread:  954576 lr:  0.000000 avg.loss:  0.161894 ETA:   0h 0m 0s


In [9]:
# 6. Test the model
model.test('imdb_valid.txt')

(5000, 0.8874, 0.8874)

## Question 4: Look at the differences between the default model and the attributes found with hyperparameters search. How do the two models differ?

The 2 models differ on the way they were treated and their results.

### TREATMENT
On question 2, we trained a FastText classifier with default parameters on the training data, and evaluated it on the test data using accuracy.

On question 3 we trained a FastText classifier with hyperparameters search functionality.
First, we split our training set into a training and a validation set. 
Then we trained the model, and in order to activate hyperparameter optimization, we provided a validation file with the autotune-validation argument. Then, fastText searches the hyperparameters that give the best f1-score on 'imdb_valid.txt'. Finaly we tested our obtained model. 

### RESULTS
The default model yielded values for precision at one as 0.5 and recall of one as 0.5 as well.

For the model with attributes found with hyperparameters search, we used the test method to evaluate the classifier on the training set that we split into a training and a validation set, which yielded values for precision at one as 0.7142 and recall of one as 0.7142 as well.

The precision is the number of the correct labels predicted by the classifier among all the labels and the recall is the number of labels successfully predicted among the real labels.

The precision and the recall of the model with attributes found with hyperparameters are 0.2142 superior than the default model. Therefore, the former model performed much better than the latter.


## Question 5: Using the tuned model, take at least 2 wrongly classified examples from the test set, and try explaining why the model failed.

Dataset will not following the correct format : `__label__<your_label> <corresponding text>`

Here we will follow the following incorrect format: `__label__<corresponding text> <your_label>`

In [10]:
df_test = lower_case_rm_punc(df_pandas_test)
np.savetxt(r'incorrect_format_1_test.txt', df_test.values, fmt='__label__%s %s')

/var/folders/st/5wgrjpj9129bgxw_hskwwwlh0000gn/T/ipykernel_60338/3809465340.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['text'] = dataframe['text'].str.replace(r'[^\w\s]+', '')


In [11]:
# Test the model with the incorrect format text
model.test('incorrect_format_1_test.txt')

(10, 0.1, 0.1)

Here we will follow the following incorrect format: `<your_label> <corresponding text>`

In [12]:
df_test = lower_case_rm_punc(df_pandas_test)
df_test_reindex=df_test.reindex(columns=["label","text"])
np.savetxt(r'incorrect_format_2_test.txt', df_test_reindex.values, fmt='%s %s')

/var/folders/st/5wgrjpj9129bgxw_hskwwwlh0000gn/T/ipykernel_60338/3809465340.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  dataframe['text'] = dataframe['text'].str.replace(r'[^\w\s]+', '')


In [13]:
# Test the model with the 2nd incorrect format text
model.test('incorrect_format_2_test.txt')

(0, nan, nan)


According to FastText's documentation, training a FastText classifier takes a text file as input and every line corresponds to a sample and must have the following format:

`__label__<your_label> <corresponding text>`

Using the tuned model, we took 2 wrongly classified examples from the test set using the incorrect following formats for the model to fail:

`__label__<corresponding text> <your_label>` (switched `<your_label>` and `<corresponding text>`)

`<your_label> <corresponding text>` (removed `__label__`)